# Quelques utilisations avancées des LED

Le [premier cahier](https://github.com/g-vidal/CahierDeProgrammes/blob/master/JeuxDeLumiere/simpleLED.ipynb) a permis de se familiariser avec le pilotage à distance par des programmes et une interface web de diodes électroluminescentes. Celui-ci aborde quelques points plus avancés tirant profit de la polarité des LEDs ou permettant de compenser l'intensité du courant insuffisante pour allumer les LEDs

## Utilisation de transistors pour augmenter l'intensité traversant une LED

Le objets connectés comme l'Edison ou la Raspberry Pi sont conçus pour être peu gourmands en électricité ce qui se traduit par des intensités plus faibles délivrées par les broches du GPIO. On se retrouve confrontés au problème inverse de celui qui faisait "griller" les LEDs parce que l'intensité délivrée par les micro-contrôleurs était trop forte. Comme il fallait ajouter un nouveau composant de protection (une résistance) il faut ajouter ici un composant supplémentaire pour rehausser l'intensité et bien sur apporter une source de courant externe. le montage utilisé est encadré sur l'image ci_dessous:

![Alimentation de deux LEDs avec transistor](images/ledboard4.png)
                        ![licence : Gérard Vidal](images/Licence.jpg)

Une alimentation 1.8V ou 3.3V (suivant la tension disponible sur l'IoT concerné) est branchée sur les fils rouge et noir, les fils de modulation broche 32 et 33 selon la même nomenclature que le premier cahier dans la continuité des deux résistances extérieiures. Le code ci-dessous permet de contrôler l'allumage et l'extincton des diodes (Leds) rouge et verte.
* importation des programmes de la bibliothèque mraa
* déclaration de l'utilisation des broches 32 et 33 GPIO12 et GPIO13

In [1]:
import mraa, time
ledg = mraa.Gpio(32)
ledr = mraa.Gpio(33)

* Choix du sens de travail de la broche *out* pour envoyer une intensité dans le circuit

In [2]:
ledg.dir(mraa.DIR_OUT)
ledr.dir(mraa.DIR_OUT)

0

Mise en place du clignotement pendant 1 minute avec une fréquence d'un clignotement par seconde pour l'une et de 2 clignotements par seconde pour l'autre

In [3]:
for i in range(0,240,1) :
    j = 2 
    k = 4 
    if (i % k == 0):
        if ((i / k) % 2 == 0):
            ledr.write(1)
        else :
            ledr.write(0)
    if (i % j == 0):
        ledg.write(1)
        time.sleep(0.25)
    else :
        ledg.write(0)
        time.sleep(0.25)


### Exercice 

Modifier l'alimentation des transistors  et observer l'effet sur les diodes (Leds). Choisir d'autres fréquences d'échantillonnage, décrire le clignotement attendu, écrire le programme correspondant

In [ ]:
for i in range(-,-,-) :
    j = - 
    k = - 
    if (i % k == -):
        
    else :


## Clignotement utilisant deux broches de contrôle et pas de masse

Comme nous l'avons vu dans les différents programmes les broches du GPIO que nous avons utilisées ont deux positions possibles soit elles envoient du courant soit leur potentiel est à 0. Si l'on connecte une diode (Led) à deux broches l'une au maximum (1) l'autre à 0 la diode (Led) va être parcourue par un courant et s'allumer. Si on permute la valeur des deux broches (Leds) la diode s'éteint mais si on permute le branchement elle se rallume. Nous allons utiliser le montage encadré ci'dessous où deux diodes (Leds) sont insérées dans la plaque en parallèle et l'une permutée par rapport à l'autre. 

![Alimentation de deux LEDs par deux broches](images/ledboard3.png)
                        ![licence : Gérard Vidal](images/Licence.jpg)
                        
Il est possible de représenter la situation sur un dessin illustrant le branchement et l'activité des diodes:

![Schéma de deux LEDs dont le branchement est permutté](images/altern2.png)
                        ![licence : Gérard Vidal](images/Licence.jpg)
                        
Nous utilisons les broches 32 et 33 l'une branchée sur le fil commun l'autre dans la continuité de la résistance. Noter qu'**il est impossible d'allumer simultanément les deux diodes**.
On utilise les déclarations précédentes des leds rouge et verte ler ledg, on met en place un clignotement symétrique de 1/10ème de seconde pour la verte et asymétrique 2/10ème-3/10ème pour la rouge.


In [4]:
for i in range(0,90,1) :
    ledr.write(0)
    for j in range(4) :
        ledg.write(0)
        time.sleep(0.1)
        ledg.write(1)
        time.sleep(0.1)
    ledg.write(0)
    for j in range(3) :
        ledr.write(1)
        time.sleep(0.3)
        ledr.write(0)
        time.sleep(0.2)

### exercice 

Mettre en place un clignotement symétrique simple des diodes (LEDs) rouge/verte. Décrire un clignotement plus complexe et le programmer.

In [ ]:
for i in range(-,-,-) :

## Comment contrôler 6 LED avec 3 fils de commande seulement ?

Dans la vie courante on est habitué à contrôler une ampoule avec deux fils, dans les exercices effectués nous avons aussi utilisé un seul fil de contrôle par LEDs et un fil de masse éventuellement commun à plusieurs LED. Dans l'exemple précédent on a vu qu'en gbranchant les deux LEDs symétriquement il était possible en permuttant la valeur des broches d'obtenir un clignotement. Il existe des dispositifs présentant de nombreuses LEDs contrôlées individuellement ou par groupes, il s'agit par exemple des chenillards lumineux dans les espaces ludiques ou des guirlandes de Noël. Malgré le grand nombre de LEDs ces dispositifs sont contrôlés par un très faible nombre de fils, comment cela est-il possible ?

Nous allons aborder un exemple basique permettant de contrôler 6 LED avec 3 fils, le procédé utilisé est le Charlie-Plexing. Ce procédé permet de contrôler avec n fils n * (n-1) LEDs; soit pour nous 3 x 2 = 6.
On utilise pour cette partie le montage encadré en rouge ci-dessous.

![CharliePlexing à 6 LED](images/ledboard6.png)
                        ![licence : Gérard Vidal](images/Licence.jpg)
                        
Les fils gris représentent les points de branchement des 3 fils de contrôle, le cablage est représenté par les fils rouges. Ce procédé repose sur la possibilité de mettre la broche non pas dans deux mais dans 3 états : 
 * état masse ..............valeur out 0
 * état alimentation........valeur out 1
 * état résistance infinie .valeur in Z
 
Le Schéma de principe du fonctionnement est donné dans l'image suivante :

![Schéma de principe du CharliePlexing à 6 LED](images/charlie6.png)
                        ![licence : Gérard Vidal](images/Licence.jpg)               

Nous représenterons chaque état par la séquence d'abbréviations la caractérisant dans l'ordre des broches 1-2-3. Par exemple 0Z1 signifie que le + est sur la broche 3 que le - est sur la broche 1 et que la broche 2 est bloquée. On a donc les états suivants
 * 10Z
 * 1Z0
 * 01Z
 * 0Z1
 * Z10
 * Z01
Cela fournit bien 6 états possibles pour notre circuit à 3 fils. Nous allons **définir** (instruction *def*) 6 ensembles d'instructions permettant chacun de réaliser l'état attendu ce sont des *fonctions* que nous nommerons du nom de l'état du circuit qu'elles provoquent. Nous en produirons une 7ème qui assurera l'enchaînement des états blink6.

In [5]:
import mraa, time

In [6]:
led1 = mraa.Gpio(23) # au centre du dispositif
led2 = mraa.Gpio(32) # 2eme broche
led3 = mraa.Gpio(33) # broche à l'extrémité
dur = 0.05

In [15]:
def etat_10Z(a,b,c) :
    led1.dir(mraa.DIR_OUT)
    led2.dir(mraa.DIR_OUT)
    led3.dir(mraa.DIR_IN)
    led1.write(1)
    led2.write(0)
    time.sleep(dur)
    return

def etat_1Z0(a,b,c) :
    led1.dir(mraa.DIR_OUT)
    led2.dir(mraa.DIR_IN)
    led3.dir(mraa.DIR_OUT)
    led1.write(1)
    led3.write(0)
    time.sleep(dur)
    return

def etat_01Z(a,b,c) :
    led1.dir(mraa.DIR_OUT)
    led2.dir(mraa.DIR_OUT)
    led3.dir(mraa.DIR_IN)
    led1.write(0)
    led2.write(1)
    time.sleep(dur)
    return

def etat_0Z1(a,b,c) :
    led1.dir(mraa.DIR_OUT)
    led2.dir(mraa.DIR_IN)
    led3.dir(mraa.DIR_OUT)
    led1.write(0)
    led3.write(1)
    time.sleep(dur)
    return

def etat_Z10(a,b,c) :
    led1.dir(mraa.DIR_IN)
    led2.dir(mraa.DIR_OUT)
    led3.dir(mraa.DIR_OUT)
    led2.write(1)
    led3.write(0)
    time.sleep(dur)
    return

def etat_Z01(a,b,c) :
    led1.dir(mraa.DIR_IN)
    led2.dir(mraa.DIR_OUT)
    led3.dir(mraa.DIR_OUT)
    led2.write(0)
    led3.write(1)
    time.sleep(dur)
    return

def etat_000(a,b,c) :
    led1.dir(mraa.DIR_IN)
    led2.dir(mraa.DIR_IN)
    led3.dir(mraa.DIR_IN)
    return


def blink6(leda,ledb,ledc) :
    for i in range(20) :
        etat_01Z(leda,ledb,ledc)
        etat_10Z(leda,ledb,ledc)
        etat_0Z1(leda,ledb,ledc)
        etat_1Z0(leda,ledb,ledc)
        etat_Z01(leda,ledb,ledc)     
        etat_Z10(leda,ledb,ledc)
        etat_000(leda,ledb,ledc)
        etat_Z10(leda,ledb,ledc)
        etat_Z01(leda,ledb,ledc)
        etat_1Z0(leda,ledb,ledc)
        etat_0Z1(leda,ledb,ledc)
        etat_10Z(leda,ledb,ledc)
        etat_01Z(leda,ledb,ledc)
    etat_000(leda,ledb,ledc)
    return

blink6(led1,led2,led3)

### Exercices

- Modifier la fréquence de balayage 
- Faire un balayage dans un seul sens
- Changer le mode de balayage (par exemple +2 -1)

# Comment contrôler 12 LED avec 4 fils de commande seulement ?

Dans un des exemples précédents on a vu qu'il était possible de contrôler 6 LED avec 3 fils, le procédé utilisé est le Charlie-Plexing. Ce procédé permet de contrôler avec n fils n * (n-1) LEDs; soit pour nous maintenant 4 x 3 = 12. Le principe de cablage est identique à celui du chenillard6.
![Schéma de principe du CharliePlexing à 12 LED](images/chen12.gif)
                        ![licence : Gérard Vidal](images/Licence.jpg)               

En ajoutant un fil on rajoute 3 combinaisons possibles dans les deux sens possibles. Toutes les LED sont implantées identiquement cathode à droite sur l'image ci-dessous. Les fils noir sont les fils de cablage du dispositif, les fils orange permettent d'ajouter de l'espace de cablâge sur la planche de développement, les cables gris sont les point de connection des broches, les câbles rouges permettent de déporterla connection des résistances.
![Chenillard à 12 LEDs](images/chenillard12.jpg)
                        ![licence : Gérard Vidal](images/Licence.jpg)               

Ce procédé repose sur la possibilité de mettre la broche non pas dans deux mais dans 3 états : 
 * état masse ..............valeur out 0
 * état alimentation........valeur out 1
 * état résistance infinie .valeur in Z
 
Nous représenterons chaque état par la séquence d'abbréviations la caractérisant dans l'ordre des broches 1-2-3-4. Par exemple 10ZZ signifie que le + est sur la broche 1, que le - est sur la broche 2, que les broches 3 et 4 sont bloquées. 
Pour fournir les 12 états possibles pour notre circuit à 4 fils, plutôt que d'énumérer les états possibles nous allons utiliser une *fonction* qui réalise systématiquement un état donné (ici 10ZZ) en permuttant les broches utilisées. Le programme principal liste les états possibles. La *fonction* est **définie** (par l'instruction *def*), elle regroupe toutes les instructions nécessaires pour mettre les broches dans l'état 10ZZ et les maintenir dans cet état pendant un intervalle de temps. Les appels successifs à cette fonction allument **une seule des diodes parmi les 12**. La permuttation des broches dans les appels permet  de réaliser l'ettet chenillard. 

In [1]:
import mraa, time, _thread

In [2]:
led1 = mraa.Gpio(23) # position 2
led2 = mraa.Gpio(32) # extérieur 1
led3 = mraa.Gpio(33) # intérieur 4
led4 = mraa.Gpio(8)  # position 3
dur = 0.025

In [4]:
# veille sur l'input
def input_thread(CheckInput):
    check = input()
#    CheckInput = CheckInput.append(check)
    CheckInput.append(check)
    return

def etat_10ZZ(a,b,c,d) :
    a.dir(mraa.DIR_OUT)
    b.dir(mraa.DIR_OUT)
    c.dir(mraa.DIR_IN)
    d.dir(mraa.DIR_IN)
    a.write(1)
    b.write(0)
    time.sleep(dur)
    return

def etat_0000(a,b,c,d) :
    for i in [a,b,c,d] :
        i.dir(mraa.DIR_IN)
    return


def blink12(leda,ledb,ledc,ledd) :
    etat_10ZZ(led3,led4,led1, led2)
    etat_10ZZ(led4,led3,led1, led2)
    etat_10ZZ(led3,led1,led2, led4)
    etat_10ZZ(led1,led3,led2, led4)
    etat_10ZZ(led4,led1,led3, led2)
    etat_10ZZ(led1,led4,led3, led2)
    etat_10ZZ(led4,led2,led1, led3)
    etat_10ZZ(led2,led4,led1, led3)
    etat_10ZZ(led1,led2,led3, led4)
    etat_10ZZ(led2,led1,led3, led4)
    etat_10ZZ(led3,led2,led1, led4)
    etat_10ZZ(led2,led3,led1, led4)
    etat_10ZZ(led2,led3,led1, led4)
    etat_10ZZ(led3,led2,led1, led4)       
    etat_10ZZ(led3,led2,led1, led4)      
    etat_10ZZ(led2,led1,led3, led4)
    etat_10ZZ(led1,led2,led3, led4)
    etat_10ZZ(led2,led4,led1, led3)
    etat_10ZZ(led4,led2,led1, led3)
    etat_10ZZ(led1,led4,led3, led2)
    etat_10ZZ(led4,led1,led3, led2)
    etat_10ZZ(led1,led3,led2, led4)
    etat_10ZZ(led3,led1,led2, led4)
    etat_10ZZ(led4,led3,led1, led2)
    etat_10ZZ(led3,led4,led1, led2)
    return

def kit_on() :
    CheckInput = []
    _thread.start_new_thread(input_thread, (CheckInput,))
    print ('Type "stop" to stop Kit')
    
    while True :
        blink12(led1,led2,led3,led4)        
        # Sortie de boucle infinie
        if CheckInput == ['stop']  : 
            break
                
            return
    
    etat_0000(led1,led2,led3,led4) 
    
    return

kit_on()

Type "stop" to stop Kit
stop
